In [1]:
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
import numpy as np
import random
from colorama import init, Fore
import collections

#import sys
#sys.setrecursionlimit(10_000)

In [2]:
class Move(Enum):
    '''
    Selects where you want to place the taken piece. The rest of the pieces are shifted
    '''
    TOP = 0
    BOTTOM = 1
    LEFT = 2
    RIGHT = 3

In [3]:
class Player(ABC):
    def __init__(self) -> None:
        '''You can change this for your player if you need to handle state/have memory'''
        pass

    @abstractmethod
    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        '''
        The game accepts coordinates of the type (X, Y). X goes from left to right, while Y goes from top to bottom, as in 2D graphics.
        Thus, the coordinates that this method returns shall be in the (X, Y) format.

        game: the Quixo game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions and a move among TOP, BOTTOM, LEFT and RIGHT
        '''
        pass

In [4]:
class Game(object):
    def __init__(self, size) -> None:
        assert size > 2
        self._board = np.ones((size, size), dtype=np.uint8) * -1
        self.current_player_idx = 0
        self.size = size
        self.num_moves = 0

    def get_board(self) -> np.ndarray:
        '''
        Returns the board
        '''
        return deepcopy(self._board)

    def get_current_player(self) -> int:
        '''
        Returns the current player
        '''
        return deepcopy(self.current_player_idx)

    #edit and added methods
    def print(self):
        '''Prints the board. -1 are neutral pieces, 0 are pieces of player 0, 1 pieces of player 1'''
        #print(self._board)

        for row in self._board:
            for cell in row:
                if cell == 0:
                    # Stampa X rossa
                    print(f"{Fore.GREEN}X{Fore.RESET}", end=" ")
                elif cell == 1:
                    # Stampa O verde
                    print(f"{Fore.RED}O{Fore.RESET}", end=" ")
                else:
                    # Stampa il valore -1 come spazio vuoto
                    print("□", end=" ")
            print()  # Vai a capo alla fine di ogni riga
     
    def check_winner(self) -> int:
        '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
        # for each row
        for x in range(self._board.shape[0]):
            # if a player has completed an entire row
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]):
                # return the relative id
                return self._board[x, 0]
        # for each column
        for y in range(self._board.shape[1]):
            # if a player has completed an entire column
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]):
                # return the relative id
                return self._board[0, y]
        # if a player has completed the principal diagonal
        if self._board[0, 0] != -1 and all(
            [self._board[x, x]
                for x in range(self._board.shape[0])] == self._board[0, 0]
        ):
            # return the relative id
            return self._board[0, 0]
        # if a player has completed the secondary diagonal
        if self._board[0, -1] != -1 and all(
            [self._board[x, -(x + 1)]
             for x in range(self._board.shape[0])] == self._board[0, -1]
        ):
            # return the relative id
            return self._board[0, -1]
        return -1

    def check_draw(self) -> bool:
        #at least one move for every tiles plus 50 in 5x5 or 40 in 4x4 and so on
        total = (self.size*self.size)+(self.size*10)
        #if there is a draw return true
        if self.num_moves >= total:
            return True
        return False

    def valid_moves(self, player_id: int) -> [ [(int,int), Move] ]:
        moves = []
        for r in range(0, self.size):
            for c in range(0, self.size):
                #check if a tile is empty or of the same type of the player
                if(self._board[r][c] == -1 or self._board[r][c] == player_id):
                    #check the up line of tiles
                    if(r == 0): 
                        moves.append( [(r,c), Move.BOTTOM ])
                        if(c != 0 and c != self.size-1):
                            moves.append( [(r,c), Move.RIGHT ])
                            moves.append( [(r,c), Move.LEFT ])
                    #check the down line of tiles
                    if(r == self.size-1):
                        moves.append( [(r,c), Move.TOP ])
                        if(c != 0 and c != self.size-1):
                            moves.append( [(r,c), Move.RIGHT ])
                            moves.append( [(r,c), Move.LEFT ])
                    #check the left line of tiles
                    if(c == 0):
                        moves.append( [(r,c), Move.RIGHT ])
                        if(r != 0 and r != self.size-1):
                            moves.append( [(r,c), Move.TOP ])
                            moves.append( [(r,c), Move.BOTTOM ])
                    #check the right line of tiles
                    if(c == self.size-1):
                        moves.append( [(r,c), Move.LEFT ])
                        if(r != 0 and r != self.size-1):
                            moves.append( [(r,c), Move.TOP ])
                            moves.append( [(r,c), Move.BOTTOM ])
        return moves                   

    def get_tiles_of_player(self, player_id) -> int:
        #count how many tiles for the specificated players simbol
        return np.count_nonzero(self._board == player_id)
                
    def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        winner = -1
        while winner < 0:
            ok = False
            while not ok:
                from_pos, slide = players[self.current_player_idx].make_move(self)
                ok = self.move(from_pos, slide, self.current_player_idx)
            #increase num moves
            self.num_moves += 1
            #only for debug print table
            #print()
            #self.print()

            #check win
            winner = self.check_winner()

            #change player turn
            self.current_player_idx = 1 - self.current_player_idx

        print("end with ", self.num_moves, " moves" )
        return winner

    def move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        '''Perform a move'''
        if player_id > 2:
            return False
        # Oh God, Numpy arrays
        prev_value = deepcopy(self._board[(from_pos[0], from_pos[1])])
        acceptable = self.__take((from_pos[0], from_pos[1]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[0], from_pos[1]), slide)
            if not acceptable:
                self._board[(from_pos[0], from_pos[1])] = deepcopy(prev_value)
        return acceptable
    
    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        '''Take piece'''
        # acceptable only if in border
        acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < self.size)
            # check if it is in the last row
            or (from_pos[0] == self.size-1 and from_pos[1] < self.size)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < self.size)
            # check if it is in the last column
            or (from_pos[1] == self.size-1 and from_pos[0] < self.size)
            # and check if the piece can be moved by the current player
        ) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable:
            self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        '''Slide the other pieces'''
        # define the corners
        last_tile = self.size-1
        SIDES = [(0, 0), (0, last_tile), (last_tile, 0), (last_tile, last_tile)]
        # if the piece position is not in a corner
        if from_pos not in SIDES:
            # if it is at the TOP, it can be moved down, left or right
            acceptable_top: bool = from_pos[0] == 0 and (
                slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is at the BOTTOM, it can be moved up, left or right
            acceptable_bottom: bool = from_pos[0] == last_tile and (
                slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is on the LEFT, it can be moved up, down or right
            acceptable_left: bool = from_pos[1] == 0 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT
            )
            # if it is on the RIGHT, it can be moved up, down or left
            acceptable_right: bool = from_pos[1] == last_tile and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT
            )
        # if the piece position is in a corner
        else:
            # if it is in the upper left corner, it can be moved to the right and down
            acceptable_top: bool = from_pos == (0, 0) and (
                slide == Move.BOTTOM or slide == Move.RIGHT)
            # if it is in the lower left corner, it can be moved to the right and up
            acceptable_left: bool = from_pos == (last_tile, 0) and (
                slide == Move.TOP or slide == Move.RIGHT)
            # if it is in the upper right corner, it can be moved to the left and down
            acceptable_right: bool = from_pos == (0, last_tile) and (
                slide == Move.BOTTOM or slide == Move.LEFT)
            # if it is in the lower right corner, it can be moved to the left and up
            acceptable_bottom: bool = from_pos == (last_tile, last_tile) and (
                slide == Move.TOP or slide == Move.LEFT)
        # check if the move is acceptable
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        # if it is
        if acceptable:
            # take the piece
            piece = self._board[from_pos]
            # if the player wants to slide it to the left
            if slide == Move.LEFT:
                # for each column starting from the column of the piece and moving to the left
                for i in range(from_pos[1], 0, -1):
                    # copy the value contained in the same row and the previous column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i - 1)]
                # move the piece to the left
                self._board[(from_pos[0], 0)] = piece
            # if the player wants to slide it to the right
            elif slide == Move.RIGHT:
                # for each column starting from the column of the piece and moving to the right
                for i in range(from_pos[1], self._board.shape[1] - 1, 1):
                    # copy the value contained in the same row and the following column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i + 1)]
                # move the piece to the right
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            # if the player wants to slide it upward
            elif slide == Move.TOP:
                # for each row starting from the row of the piece and going upward
                for i in range(from_pos[0], 0, -1):
                    # copy the value contained in the same column and the previous row
                    self._board[(i, from_pos[1])] = self._board[(
                        i - 1, from_pos[1])]
                # move the piece up
                self._board[(0, from_pos[1])] = piece
            # if the player wants to slide it downward
            elif slide == Move.BOTTOM:
                # for each row starting from the row of the piece and going downward
                for i in range(from_pos[0], self._board.shape[0] - 1, 1):
                    # copy the value contained in the same column and the following row
                    self._board[(i, from_pos[1])] = self._board[(
                        i + 1, from_pos[1])]
                # move the piece down
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable



#A basic min max approch

In [5]:
class MinMaxBase():
    def __init__(self, max_depth=4) -> None:
        self.max_depth = max_depth

    def evaluate(self, game: 'Game') -> int:
        #get my player
        my_player = game.get_current_player()
        #get enemy player
        enemy_player = 1 - my_player

        if game.check_winner() == my_player:
            return 10
        elif game.check_winner() == enemy_player:
            return -10
        else:
            #draw
            return 0

    def max(self, game: Game, depth: int, alpha: int, beta: int):
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game)
        
        best_move = None
        max_value = float('-inf')
        
        for move in game.valid_moves(game.get_current_player()):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], game_copy.get_current_player())
            game_copy.num_moves += 1
            minimum = self.min(game_copy, depth - 1, alpha, beta)
            if minimum > max_value :
                best_move = (move[0],move[1])
                max_value = minimum
            
            if max_value >= beta:
                break
            if max_value > alpha:
                alpha = max_value
        
        if depth == self.max_depth:
            return best_move

        return max_value
    
    def min(self, game: Game, depth: int, alpha: int, beta: int) -> int:
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game)
        
        best_move = None
        min_value = float('inf')

        #get other player
        enemy_player = 1-game.get_current_player()

        for move in game.valid_moves(enemy_player):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], enemy_player)
            game_copy.num_moves += 1
            maximum = self.max(game_copy, depth - 1, alpha, beta)
            if maximum < min_value:
                best_move = (move[0],move[1])
                min_value = maximum
            
            if min_value <= alpha:
                break   
            if min_value < beta:
                beta = min_value

        if depth == self.max_depth:
            return best_move

        return min_value

#A min max approch with intressing heuristics

In [6]:
class MinMaxHeuristics():
    def __init__(self, max_depth=4) -> None:
        self.max_depth = max_depth


    def evaluate(self, game: 'Game', current_depth, player_turn) -> int:
        #valutation
        evaluation = 0 
        #get my player
        my_player = game.get_current_player()
        #get enemy player
        enemy_player = 1 - my_player

        if game.check_winner() == my_player:
            return 100_000 + current_depth #current depth + 1 for avoid depth=0
        elif game.check_winner() == enemy_player:
            return -100_000 - current_depth
        else:
            #take the occurence of 4 of this player symbol
            transpose = game._board.transpose()
            my_count = []
            for row, column in zip(game._board, transpose):
                # count the occurrences of every simbol
                rowcounter = collections.Counter(row)
                columncounter = collections.Counter(column)
                #count my player simbols (get put 0 if there isn't any simbol of my type)
                my_count.append(rowcounter.get(player_turn, 0))
                my_count.append(columncounter.get(player_turn, 0))
            #check for only 4 occurency in horizontal and vertical
            occValue = my_count.count(4) * 5
            #negativize if is enemy turn
            if(player_turn == enemy_player):
                occValue *= -1

            #check for the center piece
            if(game._board[2,2] == my_player ):
                evaluation += 20
            elif(game._board[2,2] == enemy_player):
                evaluation -= 20

            score_my_count_tiles = game.get_tiles_of_player(my_player)
            score_enemy_count_tiles =  game.get_tiles_of_player(enemy_player)

            evaluation += (score_my_count_tiles - score_enemy_count_tiles) + occValue
            return evaluation 

    def max(self, game: Game, depth: int, alpha: int, beta: int):
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game, depth, game.get_current_player())

        best_move = None
        max_value = float('-inf')
        
        for move in game.valid_moves(game.get_current_player()):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], game_copy.get_current_player())
            game_copy.num_moves += 1
            minimum = self.min(game_copy, depth - 1, alpha, beta)
            if minimum > max_value :
                best_move = (move[0],move[1])
                max_value = minimum
            
            if max_value >= beta:
                break
            if max_value > alpha:
                alpha = max_value
        
        if depth == self.max_depth:
            return best_move

        return max_value
    
    def min(self, game: Game, depth: int, alpha: int, beta: int) -> int:
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game, depth, 1-game.get_current_player())
        
        best_move = None
        min_value = float('inf')

        #get other player
        enemy_player = 1-game.get_current_player()

        for move in game.valid_moves(enemy_player):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], enemy_player)
            game_copy.num_moves += 1
            maximum = self.max(game_copy, depth - 1, alpha, beta)
            if maximum < min_value:
                best_move = (move[0],move[1])
                min_value = maximum
            
            if min_value <= alpha:
                break   
            if min_value < beta:
                beta = min_value

        if depth == self.max_depth:
            return best_move

        return min_value

#A min max approch with a better heuristics

In [7]:
class MinMaxHeuristics2():
    def __init__(self, max_depth=4) -> None:
        self.max_depth = max_depth

    def evaluate(self, game: 'Game', current_depth) -> int:
        #valutation
        evaluation = 0 
        #get my player
        my_player = game.get_current_player()
        #get enemy player
        enemy_player = 1 - my_player

        if game.check_winner() == my_player:
            evaluation = 100_000*(current_depth + 1) #current depth + 1 for avoid depth=0
        elif game.check_winner() == enemy_player:
            evaluation = -100_000*(current_depth + 1)
        else:
            #no victory for either player, so I check at least the number of tiles per player
            evaluation = game.get_tiles_of_player(my_player) - game.get_tiles_of_player(enemy_player)

        #heuristics for generale case (no win no lose)
        transpose = game._board.transpose()		# columns in state.board = rows in transpose
        my_count = []
        enemy_count = []
        for row, column in zip(game._board, transpose):
            # count the occurrences of every simbol
            rowcounter = collections.Counter(row)
            columncounter = collections.Counter(column)
            #count my player simbols (get put 0 if there isn't any simbol of my type)
            my_count.append(rowcounter.get(my_player, 0))
            my_count.append(columncounter.get(my_player, 0))
            #count enemy plyer simbols
            enemy_count.append(rowcounter.get(enemy_player, 0))
            enemy_count.append(columncounter.get(enemy_player, 0))

        #get mirrored on diagonal board
        mirror_diagonal = game._board[:, ::-1]
        #put diagonals in an array
        diagonals = [np.diagonal(game._board), np.diagonal(mirror_diagonal)]
        # count the occurrences of every simbol
        main_diagonal_count = collections.Counter(diagonals[0])
        second_diagonal_count = collections.Counter(diagonals[1])
        # count my player simbols in diagonals
        my_count.append(main_diagonal_count.get(my_player, 0))
        my_count.append(second_diagonal_count.get(my_player, 0))
        #count enemy player simbols in diagonals
        enemy_count.append(main_diagonal_count.get(enemy_player, 0))
        enemy_count.append(second_diagonal_count.get(enemy_player, 0))

        #We want give more poins for the player that take more blanck tile
        score_my_count_tiles = game.get_tiles_of_player(my_player) * 10_000
        score_enemy_count_tiles = game.get_tiles_of_player(enemy_player) * 10_000

        score_my_count_combo = 0
        score_enemy_count_combo = 0

        #give different poins for different number of symbol for each row,column,diagonal
        for x in my_count:
            score_my_count_combo += 5 ** x

        for y in enemy_count:
            score_enemy_count_combo += 5 ** y
		
        my_score = score_my_count_tiles + score_my_count_combo
        enemy_score = score_enemy_count_tiles + score_enemy_count_combo

        final_score = my_score-enemy_score

        return (evaluation + final_score)
            

    def max(self, game: Game, depth: int, alpha: int, beta: int):
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game, depth)
        
        best_move = None
        max_value = float('-inf')
        
        for move in game.valid_moves(game.get_current_player()):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], game_copy.get_current_player())
            game_copy.num_moves += 1
            minimum = self.min(game_copy, depth - 1, alpha, beta)
            if minimum > max_value :
                best_move = (move[0],move[1])
                max_value = minimum
            
            if max_value >= beta:
                break
            if max_value > alpha:
                alpha = max_value
        
        if depth == self.max_depth:
            return best_move

        return max_value
    
    def min(self, game: Game, depth: int, alpha: int, beta: int) -> int:
        if depth == 0 or game.check_winner() != -1:
            return self.evaluate(game, depth)
        
        best_move = None
        min_value = float('inf')

        #get other player
        enemy_player = 1-game.get_current_player()

        for move in game.valid_moves(enemy_player):
            game_copy = deepcopy(game)
            game_copy.move(move[0], move[1], enemy_player)
            game_copy.num_moves += 1
            maximum = self.max(game_copy, depth - 1, alpha, beta)
            if maximum < min_value:
                best_move = (move[0],move[1])
                min_value = maximum
            
            if min_value <= alpha:
                break   
            if min_value < beta:
                beta = min_value

        if depth == self.max_depth:
            return best_move

        return min_value

In [ ]:
#ex main 
class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:

        last_tile = game.size-1
        from_pos = (random.randint(0, last_tile), random.randint(0, last_tile))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move
        
class HumanPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        px = int(input('Insert the X coordinate: '))
        py = int(input('Insert the Y coordinate: '))
        pmove = Move( int(input('Insert the Move (TOP=0 BOTTOM=1 LEFT=2 RIGHT=3 : ')) )
        return (px, py), pmove

class MinMaxBasePlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        my_depth = 4
        #crete minmax class with my depth
        minmax = MinMaxBase(my_depth)
        #calling method to get the best move in this sitation
        from_pos, move = minmax.max(game, my_depth, -1_000_000, 1_000_000)
        return from_pos, move

class MinMaxHeuristicsPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        my_depth = 4
        #crete minmax class with my depth
        minmax = MinMaxHeuristics(my_depth)
        #calling method to get the best move in this sitation
        from_pos, move = minmax.max(game, my_depth, -1_000_000, 1_000_000)
        return from_pos, move
    
class MinMaxHeuristics2Player(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        my_depth = 4
        #crete minmax class with my depth
        minmax = MinMaxHeuristics2(my_depth)
        #calling method to get the best move in this sitation
        from_pos, move = minmax.max(game, my_depth, -1_000_000, 1_000_000)
        return from_pos, move


if __name__ == '__main__':
    win_p1 = 0
    win_p2 = 0
    draw   = 0
    while(True):
        game_size = 5
        g = Game(game_size)
        g.print()
        player1 = MinMaxBasePlayer() #MyPlayer(game_size) #IA, X -> 0
        player2 = RandomPlayer() # random, O -> 1
        winner = g.play(player1, player2)
        g.print()
        print(f"Winner: Player {winner}")

        if(winner == 0): win_p1+=1
        if(winner == 1): win_p2+=1
        if(winner == -1): draw+=1
        print("\n\n",win_p1,win_p2, draw,"\n\n")